EP02.1 - Visão e Processamento de Imagens
===

Nº USP | Nome
---|---|
11347794|Aarão Melo Lopes|

---

## Acesso


Se estiver utilizando o Google Colaboratory execute os passos a seguir. No entanto, se estiver rodando em um dispositivo local, ignore esta seção.


Realize login com sua conta google, para ter acesso aos arquivos compartilhados, executando a célula abaixo.

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Mude o diretório corrente para a pasta do projeto.

In [41]:
cd drive/Shared drives/EP02 - Visão e Processamento de Imagens

[Errno 2] No such file or directory: 'drive/Shared drives/EP02 - Visão e Processamento de Imagens'
/content/drive/Shared drives/EP02 - Visão e Processamento de Imagens


## Geração



A função abaixo carrega as imagens de um diretório em uma matriz.

In [ ]:
from skimage.io import imread, imsave
def loadDir(path):
  dataset = []
  for i in range(1080):
    dataset.append(imread(path+"/" + str(i+1) + '.jpeg'))
    print('read ' + str(i+1) + '.jpeg')
  return dataset

### Escala de Cinza

Execute a célula abaixo para criar e salvar as images em escala de cinza

In [ ]:
pip install imgaug

In [ ]:
import imgaug.augmenters as iaa
grayScale = iaa.Grayscale(alpha=1.0).augment_images(dataset)
saveDir(grayScale, 'augmentedDataset/grayScale')

In [ ]:
grayScale = loadDir('augmentedDataset/grayScale')

### Sombreamento

In [ ]:
def darkImage(): return [[[0, 0, 0] for j in range(720)] for i in range(1280)]

In [ ]:
import numpy as np
def gradient():
  result = darkImage()
  for i in range(1280):
    for j in range(720):
      x = (j+0.5)/720-0.5
      y = (i+0.5)/1280-0.5
      color =  int(((x**2+y**2)**0.5/(2**0.5))**0.7*256)
      result[i][j] = [color, color, color]
  return np.uint8(result)

In [ ]:
import numpy as np
def add(image1, image2):
  result = darkImage()
  for i in range(1280):
    for j in range(720):
      color = min(255,int(image1[i][j][0]) + int(image2[i][j][0]))
      result[i][j] = [color, color, color]
  return np.uint8(result) 

In [ ]:
from skimage.io import imread, imsave
g = gradient()
for i in range(1080):
  im = imread("augmentedDataset/grayScale/" + str(i+1) + '.jpeg')
  sdw = add(im, g )
  del im
  imsave('augmentedDataset/shadowScale/'+str(i+1) + '.jpeg', sdw)
  del sdw
  print("save shadow " + str(i+1) )

### Logarítmo

$$s=c\cdot{\rm log}(1+r)$$

$$c=\frac{L-1}{{\rm log}L}$$



In [ ]:
from math import log
import numpy as np

def logarithm(image):
  logarithm = darkImage()
  for i in range(1280):
    for j in range(720):
      r = image[i][j][0]
      color = round(255/log(256)*log(1+r))
      logarithm[i][j]= [color,color,color]
  return np.uint8(logarithm)

In [ ]:
from skimage.io import imread, imsave
for i in range(1080):
  im = imread("augmentedDataset/grayScale/" + str(i+1) + '.jpeg')
  lgrth = logarithm(im)
  del im
  imsave('augmentedDataset/logScale/'+str(i+1) + '.jpeg', lgrth)
  del lgrth
  print("save log " + str(i+1) )

### Potência

$$s=c\cdot r^{\gamma}$$

$$c=(L-1)^{1-\gamma}$$

In [ ]:
import numpy as np

def pow(image):
  pow = darkImage()
  gamma = 2.7182818284590452353602874
  for i in range(1280):
    for j in range(720):
      r = image[i][j][0]
      color = round(255**(1-gamma)*r**gamma)
      pow[i][j]= [color,color,color]
  return np.uint8(pow)

In [ ]:
from skimage.io import imread, imsave
for i in range(1080):
  im = imread("augmentedDataset/grayScale/" + str(i+1) + '.jpeg')
  pw = pow(im)
  del im
  imsave('augmentedDataset/powScale/'+str(i+1) + '.jpeg', pw)
  del pw
  print("save pow " + str(i+1) )

### Filtro da Média

In [ ]:
def zeroPadding(image):
  zeroPadding = darkImage()
  for i in range(1280):
    for j in range(720):
      zeroPadding[i+1][j+1] = image[i][j]
  return zeroPadding

In [ ]:
import numpy as np
def meanFilter(image):
  mean =  darkImage()
  imz = zeroPadding(image)
  for i in range(1280):
    for j in range(720):
      color = 0
      for a in [-1,0,1]:
        for b in [-1,0,1]:
          color += imz[i+1+a][j+1+b][0]
      color /= 9
      mean[i][j] = [color, color, color]
  return np.uint8(mean)

In [ ]:
from skimage.io import imread, imsave
for i in range(1080):
  im = imread("augmentedDataset/grayScale/" + str(i+1) + '.jpeg')
  mean = meanFilter(im)
  del im
  imsave('augmentedDataset/meanScale/'+str(i+1) + '.jpeg', mean)
  del mean
  print("save mean " + str(i+1) )

## Visualização


Visualização dos dados aumentados

In [ ]:
import matplotlib.pyplot as plt
from skimage.io import imread, imshow, imsave

def showimages(image_list):
  for j in range(40):
    fig, axes = plt.subplots(1, 27, figsize=(20, 20))
    ax = axes.ravel()
    for i in range(27):
        ax[i].imshow(image_list[i+j*27])
        ax[i].axis('off')
    fig.tight_layout()
    plt.show()
    
def loadDir(path):
  dataset = []
  for i in range(1080):
    dataset.append(imread(path+"/" + str(i+1) + '.jpeg'))
    print('read ' + str(i+1) + '.jpeg')
  return dataset

In [ ]:
grayScale = loadDir("augmentedDataset/grayScale")

In [ ]:
showimages(grayScale)

In [ ]:
powScale = loadDir("augmentedDataset/powScale")

In [ ]:
showimages(powScale)

In [ ]:
shadowScale = loadDir("augmentedDataset/shadowScale")

In [ ]:
showimages(shadowScale)

In [ ]:
logScale = loadDir("augmentedDataset/logScale")

In [ ]:
showimages(logScale)